# Project for Understanding Class

We need to design an implement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)


For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

## Possible Approach

- Create two classes: `TimeZone` to store the time zone name and offset definition(in hours and minutes) and a Main Class `Account` that will have the following `public` interface:
    - initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
    - a first name property (read/write)
    - a last name property (read/write)
    - a full name property (computed, read-only)
    - a balance property (read-only)
    - an interest rate property (class level property)
    - deposit, withdraw, pay_interest methods
    - parse confirmation code

In [7]:
import itertools
import numbers
from datetime import timedelta, datetime

In [3]:
# Build a TimeZone Class
class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('TimeZone name cannot be empty.')
        
        self._name = str(name).strip()

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError("Offset hours must be an Integer.")
        
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Offset Minutes must be an Integer.')
        
        if offset_minutes < -59 or offset_minutes > 59:
            raise ValueError('Offset Minutes must be in range -59 to 59.(inclusive)')
        
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)

        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be in range -12:00 to 14:00')
        
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset
    
    @property
    def name(self):
        return self._name
    
    def __eq__(self, other):
        return (isinstance(other, TimeZone) and 
                self.name == other.name and
                self._offset_hours == other._offset_hours and 
                self._offset_minutes == other._offset_minutes)
    
    def __repr__(self):
        return (f"TimeZone(name='{self.name}', "
                f"Offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes})")

In [4]:
tz1 = TimeZone(name='ist', offset_hours=5, offset_minutes=30)

In [5]:
tz1

TimeZone(name='ist', Offset_hours=5, offset_minutes=30)

In [6]:
tz1 = TimeZone(name='abc', offset_hours=15, offset_minutes=30)

ValueError: Offset must be in range -12:00 to 14:00

In [18]:
from collections import namedtuple

Confirmation = namedtuple('Confirmation', 'account_number, transaction_code, transaction_id, time_utc, time')

In [19]:
# Build Account Class
class Account:
    transaction_counter = itertools.count(100)
    _interest_rate = 0.5

    _transaction_codes = {
        'deposit': 'D', 
        'withdraw': 'W', 
        'interest': 'I',
        'rejected': 'X'
    }

    def __init__(self, account_number, first_name, last_name, timezone=None, initial_balance=0):
        self._account_number = account_number
        self._first_name = first_name
        self._last_name = last_name

        if timezone is None:
            timezone = TimeZone(name='UTC', offset_hours=0, offset_minutes=0)
        self._timezone = timezone

        self._balance = Account.validate_real_number(initial_balance)

    @property
    def account_number(self):
        return self._account_number
    
    @property
    def timezone(self):
        return self._timezone
    
    @property
    def first_name(self):
        return self._first_name
    
    @property
    def last_name(self):
        return self._last_name
    
    @first_name.setter
    def first_name(self, value):
        self.validate_and_set_name(property_name='_first_name', value=value, field_title='First Name')

    @last_name.setter
    def last_name(self, value):
        self.validate_and_set_name(property_name='_last_name', value=value, field_title='Last Name')

    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    @property
    def balance(self):
        return self._balance

    def validate_and_set_name(self, property_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f"{field_title} cannot be empty.")
        setattr(self, property_name, value)

    def generate_confirmation_code(self, transaction_code):
        dt_str = datetime.now().strftime("%Y%m%d%H%M%S")
        return f'{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_counter)}'
    
    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone=None):
        # D-140568-20190315145900-124
        parts = confirmation_code.split('-')
        if len(parts) != 4:
            raise ValueError('Invalid confirmation code')
        
        # unpack
        transaction_code, account_number, raw_dt_utc, transaction_id = parts

        try:
            dt_utc = datetime.strptime(raw_dt_utc, "%Y%m%d%H%M%S")
        except ValueError as err:
            raise ValueError('Invalid transaction datetime') from err
        
        if preferred_time_zone is None:
            preferred_time_zone = TimeZone(name='UTC', offset_hours=0, offset_minutes=0)

        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError('Invalid TimeZone specified')
        
        dt_preferred = dt_utc + preferred_time_zone.offset
        dt_preferred_str = f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_time_zone.name})"

        return Confirmation(account_number, transaction_code, transaction_id, dt_utc.isoformat(), dt_preferred_str)

    @staticmethod
    def validate_real_number(value, min_value=None):
        if not isinstance(value, numbers.Real):
            raise ValueError('Value must be real number')
        
        if min_value is not None and value < min_value:
            raise ValueError(f'Value must be at least {min_value}')
        
        return value

    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a real number.')
        if value < 0:
            raise ValueError('interest rate must not be negative.')
        cls._interest_rate = value

    def deposit(self, value):
        value = Account.validate_real_number(value=value, min_value=0.01)

        # Get transaction code
        transaction_code = Account._transaction_codes['deposit']

        # Generate confirmation code
        conf_code = self.generate_confirmation_code(transaction_code=transaction_code)

        # Make deposit
        self._balance += value
        return conf_code
    
    def withdraw(self, value):
        value = Account.validate_real_number(value=value, min_value=0.01)
        accepted = False
        if self.balance - value < 0:
            transaction_code = Account._transaction_codes['rejected']
        else:
            transaction_code = Account._transaction_codes['withdraw']
            accepted = True
        
        conf_code = self.generate_confirmation_code(transaction_code=transaction_code)

        if accepted:
            self._balance -= value

        return conf_code
    
    def pay_interest(self):
        interest = self.balance * Account.get_interest_rate() / 100
        conf_code = self.generate_confirmation_code(Account._transaction_codes['interest'])
        self._balance += interest
        return conf_code

In [20]:
a = Account(account_number='A100', first_name='Himanshu', last_name='getLost', timezone=TimeZone('IST', 5, 30), initial_balance=100)

In [21]:
print(a.balance)
print(a.deposit(150))
print(a.balance)
print(a.withdraw(50))
print(a.balance)

100
D-A100-20240924135220-100
250
W-A100-20240924135220-101
200


In [22]:
Account.set_interest_rate(1.0)

In [23]:
print(a.get_interest_rate())

1.0


In [24]:
print(a.pay_interest())

I-A100-20240924135338-102


In [25]:
print(a.balance)

202.0


In [26]:
Account.parse_confirmation_code(confirmation_code='D-A100-20240924135220-100', preferred_time_zone=TimeZone('IST', 5, 30))

Confirmation(account_number='A100', transaction_code='D', transaction_id='100', time_utc='2024-09-24T13:52:20', time='2024-09-24 19:22:20 (IST)')